## Imports and dependencies

We recommend you install `Anaconda Python` (2.7). Please also ensure you have have psycopg2 installed & configured for your operating system.

In [6]:
%matplotlib inline
import psycopg2
import pandas as pd
import pandas.io.sql as psql
import ConfigParser
import os
import IPython
from IPython.display import display
from IPython.display import HTML
#Maximum height of a result set
pd.set_option('display.max_rows', 500)
#Maximum number of rows to display in a cell.
pd.set_option('display.max_columns', 100)
#Show the full content of columns
pd.set_option('display.max_colwidth', -1)

## Setting up database connectivity

1. Create a file in your home directory containing the database credentials for your target environment. 
For example:
```
~ vim ~/.dslab_user.cred
```
The content of this file should look like so (with appropriate values for HOSTNAME, PORT, USER, DATABASE & PASSWORD).
```
[database_creds]
host: HOSTNAME
port: PORT
user: USER
database: DATABASE
password: PASSWORD
```
2. Please set the permissions of this file to u+rwx (700), so that only you can access this file.
```
~ chmod 700 ~/.dslab_user.cred
```
You should see the following
```
~ ls -l ~/.dslab_user.cred 
-rwx------  1 USER  720748206  93 Jun 29 17:27 $HOME/.dslab_user.cred
```

## Creating database connection string

In [9]:
USER_CRED_FILE = os.path.join(os.path.expanduser('~'), '.dslab_user.cred')
def fetchDBCredentials(dbcred_file=USER_CRED_FILE):
    """
       Read database access credentials from the file in $HOME/.training.cred
    """
    #Read database credentials from user supplied file
    conf = ConfigParser.ConfigParser()
    conf.read(dbcred_file)
    #host, port, user, database, password
    host = conf.get('database_creds','host')
    port = conf.get('database_creds','port')
    user = conf.get('database_creds','user')
    database = conf.get('database_creds','database')
    password = conf.get('database_creds','password')

    #Initialize connection string
    conn_str =  """dbname='{database}' user='{user}' host='{host}' port='{port}' password='{password}'""".format(                       
                    database=database,
                    host=host,
                    port=port,
                    user=user,
                    password=password
            )
    return conn_str

## Create a connection object to the database

In [10]:
conn = psycopg2.connect(fetchDBCredentials(USER_CRED_FILE))

## Define magic commands to run SQL inline

In [11]:
from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)

@register_cell_magic
def showsql(line, cell):
    """
        Extract the code in the specific cell (should be valid SQL), and execute
        it using the connection object to the backend database. 
        The resulting Pandas dataframe
        is rendered inline below the cell using IPython.display.
        You'd use this for SELECT
    """
    #Use the global connection object defined above.
    global conn
    _ = psql.read_sql(cell, conn)
    conn.commit()
    display(_)
    return
    
@register_cell_magic
def execsql(line, cell):
    """
        Extract the code in the specific cell (should be valid SQL), and execute
        it using the connection object to the backend database. 
        You'd use this for CREATE/UPDATE/DELETE
    """
    #Use the global connection object defined above.
    global conn
    _ = psql.execute(cell, conn)
    conn.commit()
    return

# We delete these to avoid name conflicts for automagic to work
del execsql, showsql

## Test your connection to the database

#### Using the `showsql` magic command we defined above

```
%%showsql
select
    random() as x,
    random() as y
from
    generate_series(1, 5);
```

#### Alternatively you could also use plain-old pandas if you'd like to get a dataframe returned

```
df = psql.read_sql("""select random() as x, random() as y from generate_series(1, 10) q;""", conn)
df.head()
```

If you see an HTML table of the result set above, your connection to the database was successful. 

## Opening Connections to Multiple Clusters

If you want to open multiple connections (say one for GPDB and for your HAWQ cluster), you can create another file similar to `~/.dslab_user.cred`, populate the appropriate credentials, and supply this file as input to the `fetchDBCredentials()` function shown above.

For instance, let's say you created another file `~/.dslab_user.cred.gpdb` containing the appropriate credentials to connect to your GPDB cluster, then you can open a connection to this cluster with psycopg2 as follows:

```
db_cred_file_gpdb = os.path.join(os.path.expanduser('~'), '.dslab_user.cred.gpdb')
conn_gpdb = psycopg2.connect(fetchDBCredentials(db_cred_file_gpdb))
df = psql.read_sql("""select random() as x, random() as y from generate_series(1, 10) q;""", conn_gpdb)
df.head()
```

For all subsequent instance where you want to query the data on GPDB, you can use `conn_gpdb` in place of `conn` in your code.

In [19]:
%%showsql
select uid, latitude, longitude, altitude, epoch, tdate, ttime, st_point(latitude,longitude) as blocation from geolife limit 10;

,uid,latitude,longitude,altitude,epoch,tdate,ttime,blocation
0,001,39.984094,116.319236,492.0,39744.245197,2008-10-23,05:53:05,01010000004C18CDCAF6FD43402DEDD45C6E145D40
1,001,39.984198,116.319322,492.0,39744.245208,2008-10-23,05:53:06,0101000000F1643733FAFD43402CB98AC56F145D40
2,001,39.984224,116.319402,492.0,39744.245266,2008-10-23,05:53:11,01010000001AF8510DFBFD4340BA11161571145D40
3,001,39.984211,116.319389,492.0,39744.245324,2008-10-23,05:53:16,010100000086AE44A0FAFD4340F06C8FDE70145D40
4,001,39.984217,116.319422,491.0,39744.245382,2008-10-23,05:53:21,0101000000689599D2FAFD4340DEE7F86871145D40
5,001,39.984710,116.319865,320.0,39744.245405,2008-10-23,05:53:23,0101000000A7052FFA0AFE4340A4880CAB78145D40
6,001,39.984674,116.319810,325.0,39744.245463,2008-10-23,05:53:28,01010000005A9C31CC09FE4340C3BB5CC477145D40
7,001,39.984623,116.319773,326.0,39744.245521,2008-10-23,05:53:33,0101000000D8F15F2008FE434034492C2977145D40
8,001,39.984606,116.319732,327.0,39744.245579,2008-10-23,05:53:38,010100000002B9C49107FE434005DF347D76145D40
9,001,39.984555,116.319728,324.0,39744.245637,2008-10-23,05:53:43,0101000000800EF3E505FE434064E76D6C76145D40


In [37]:
%%showsql
select tdate, array_agg(distinct(uid)) from geolife_wlb_location where tdate between '2007-01-01' and '2007-12-31' group by tdate order by tdate;

,tdate,array_agg
0,2007-04-12,"[097, 142, 161, 163]"
1,2007-04-13,"[111, 142, 161, 163]"
2,2007-04-14,"[111, 128, 142, 161]"
3,2007-04-15,"[097, 111, 128, 142]"
4,2007-04-16,"[076, 097, 111, 128, 142, 161, 163]"
5,2007-04-17,"[076, 097, 111, 142, 161]"
6,2007-04-18,"[076, 097, 111, 128, 142]"
7,2007-04-19,"[076, 097, 111, 128, 161]"
8,2007-04-20,"[076, 097, 111, 128, 142, 161]"
9,2007-04-21,"[097, 111, 128, 142]"


In [38]:
%%showsql
select tdate, array_agg(distinct(uid)) from geolife_wlb_location where tdate between '2008-01-01' and '2008-12-31' group by tdate order by tdate;


,tdate,array_agg
0,2008-01-01,"[051, 062, 091, 092, 104, 181]"
1,2008-01-02,"[062, 104]"
2,2008-01-03,"[062, 091, 104, 134]"
3,2008-01-04,"[062, 091, 104, 134]"
4,2008-01-05,"[062, 091, 092, 099, 110, 134]"
5,2008-01-06,[091]
6,2008-01-07,"[051, 062, 091, 099, 110, 134]"
7,2008-01-08,"[051, 062, 091]"
8,2008-01-09,"[091, 092, 101, 104, 134]"
9,2008-01-10,"[062, 091, 092, 101, 104]"


In [39]:
%%showsql
select tdate, array_agg(distinct(uid)) from geolife_wlb_location where tdate between '2009-01-01' and '2009-12-31' group by tdate order by tdate;


,tdate,array_agg
0,2009-01-01,"[002, 003, 004, 025, 068, 085, 126, 128, 135, 144, 153, 167]"
1,2009-01-02,"[003, 004, 025, 068, 085, 128, 135, 144, 153]"
2,2009-01-03,"[003, 004, 010, 025, 068, 128, 135, 144, 153]"
3,2009-01-04,"[003, 004, 025, 085, 128, 140, 144, 153]"
4,2009-01-05,"[003, 004, 017, 025, 068, 085, 128, 140, 144]"
5,2009-01-06,"[003, 004, 017, 025, 085, 126, 128, 140, 153, 167]"
6,2009-01-07,"[003, 004, 017, 025, 033, 068, 085, 128, 140, 153]"
7,2009-01-08,"[002, 003, 004, 017, 025, 068, 083, 085, 128, 140, 144, 153]"
8,2009-01-09,"[002, 003, 004, 010, 014, 017, 022, 024, 025, 033, 068, 083, 085, 128, 135, 140, 144, 153]"
9,2009-01-10,"[002, 003, 004, 017, 022, 024, 025, 033, 083, 085, 128, 135, 140, 144, 153]"
